# Import liabraries 

In [ ]:
import os
from os import listdir, makedirs
from os.path import join, exists, expanduser
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import tensorflow as tf
from keras.utils.data_utils import Sequence
import sys
from PIL import *
from tensorflow.keras.applications import ResNet50V2
sys.modules['Image'] = Image 

# Load Data

In [ ]:
#downloaded dataset to pc
#add your paths for data location when running 
#edit paths if not running from muhammadpc

train_path_of_data = "Documents/muhammadpc/kaggle/input/chest-xray-pneumonia/chest_xray/train/"
val_path_of_data = "Documents/muhammadpc/kaggle/input/chest-xray-pneumonia/chest_xray/val/"
test_path_of_data= "Documents/muhammadpc/kaggle/input/chest-xray-pneumonia/chest_xray/test/"

In [ ]:
print(train_path_of_data)
print(val_path_of_data)
print(test_path_of_data)


In [ ]:
#load images 
normal_images = glob.glob(train_path_of_data+"NORMAL/*.jpeg")
pneumonia_images = glob.glob(train_path_of_data+"PNEUMONIA/*.jpeg")

In [ ]:
total_number_of_normal_images= len(normal_images)
total_number_of_pneumonia_images = len(pneumonia_images)
print("number of normal images=", total_number_of_normal_images)
print("number of pneumonia images=", total_number_of_pneumonia_images)

# Data Analysis and visualization

In [ ]:
#plot distribution
sns.set_theme(style="whitegrid")
plt.figure(figsize = (8,8))
ax = sns.barplot( x = ["Normal", "Pneumonia"], y = [total_number_of_normal_images, total_number_of_pneumonia_images])

In [ ]:
#plot images and resize them 

fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(15,10), subplot_kw={'xticks':[], 'yticks':[]})
for i, ax in enumerate(axes.flat):
    img = cv2.imread(normal_images[i])
    img = cv2.resize(img, (220,220))
    ax.imshow(img)
    ax.axis("off")
    ax.set_title("Normal Image")
    
    
plt.show()

fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(15,10), subplot_kw={'xticks':[], 'yticks':[]})
for i, ax in enumerate(axes.flat):
    img = cv2.imread(pneumonia_images[i])
    img = cv2.resize(img, (220,220))
    ax.imshow(img)
    ax.axis("off")
    ax.set_title("Pneumonia Image")
fig.tight_layout()    
plt.show()

# Data Augmentation/Preprocess

In [ ]:
#normalize
train_val_generator = ImageDataGenerator(rescale=1./255,horizontal_flip=True,zoom_range=0.3)
test_generator = ImageDataGenerator(rescale=1./255)

In [ ]:
#color mode=greyscale to preprocess all images to grayscale
train = train_val_generator.flow_from_directory(train_path_of_data,
                                               batch_size=128,
                                               target_size=(220,220),
                                               color_mode="grayscale",
                                               class_mode="binary",
                                               shuffle=True,
                                               seed=42,
                                               subset="training")

In [ ]:
#color mode=greyscale to preprocess all images to grayscale
val = train_val_generator.flow_from_directory(val_path_of_data,
                                               batch_size=4,
                                               target_size=(220,220),
                                               color_mode="grayscale",
                                               class_mode="binary",
                                               shuffle=True,
                                               seed=42)

In [ ]:
#color mode=greyscale to preprocess all images to grayscale
test = test_generator.flow_from_directory(test_path_of_data,
                                          batch_size=32,
                                          target_size=(220,220),
                                          color_mode="grayscale",
                                          class_mode="binary")

# Load Model 

can get model from keras function docs https://keras.io/api/applications/resnet/

In [ ]:
resnet50 = ResNet50V2(weights = "imagenet", input_shape = (220,220,3), include_top = False)
for layer in resnet50.layers:
    layer.trainable = False

model = Sequential()
model.add(resnet50)

for layer in resnet50.layers:
    layer.trainable = False
    
model.add(Flatten())
model.add(Dense(units = 128, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(units = 1, activation = "sigmoid"))
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
model.summary()

In [ ]:
model.summary()

In [ ]:
#tested both early stopping vs dropout(dropout gave better results therfore removed early stopping from code and kept dropout)


# Plot model

In [ ]:
hist = model.fit(train,validation_data=val,epochs=60)

# Evaluate model

In [ ]:
plt.figure(figsize=(8,8))
plt.plot(hist.history['loss'], color='b', label="Training loss")
plt.plot(hist.history['val_loss'], color='r', label="Validation loss")
plt.legend()
plt.show()

plt.figure()

plt.figure(figsize=(8,8))
plt.plot(hist.history['accuracy'], color='b', label="Training accuracy")
plt.plot(hist.history['val_accuracy'], color='r',label="Validation accuracy")
plt.legend(loc = "lower right")
plt.show()